In [ ]:
pip install dlib

In [ ]:
!kaggle datasets files -d rishab260/uta-reallife-drowsiness-dataset


Next Page Token = CfDJ8PHSCL9k9s1HuJ2cRFBFhugZPNgIIMozRUmO7IeKz3mXA_biqS2WJrXNlOElAtBgu2E1M0uexGY1sTSb2MFu3gQ
name                                size  creationDate         
---------------------------------  -----  -------------------  
Fold1_part1/Fold1_part1/01/0.mov     1GB  2023-01-25 09:40:42  
Fold1_part1/Fold1_part1/01/10.MOV  440MB  2023-01-25 09:40:17  
Fold1_part1/Fold1_part1/01/5.mov   453MB  2023-01-25 09:40:20  
Fold1_part1/Fold1_part1/02/0.mov   771MB  2023-01-25 09:40:46  
Fold1_part1/Fold1_part1/02/10.MOV    1GB  2023-01-25 09:41:51  
Fold1_part1/Fold1_part1/02/5.MOV     1GB  2023-01-25 09:41:24  
Fold1_part1/Fold1_part1/03/0.MOV     1GB  2023-01-25 09:39:12  
Fold1_part1/Fold1_part1/03/10.mov  576MB  2023-01-25 09:38:11  
Fold1_part1/Fold1_part1/03/5.mov   577MB  2023-01-25 09:38:20  
Fold1_part1/Fold1_part1/04/0.mp4    61MB  2023-01-25 09:39:49  
Fold1_part1/Fold1_part1/04/10.mp4   60MB  2023-01-25 09:39:53  
Fold1_part1/Fold1_part1/04/5.mp4   667MB  2023-01-25 09:39

In [ ]:
!kaggle datasets download -d rishab260/uta-reallife-drowsiness-dataset -f "Fold1_part1/Fold1_part1/01/0.mov"


Dataset URL: https://www.kaggle.com/datasets/rishab260/uta-reallife-drowsiness-dataset
License(s): CC0-1.0
 99% 1.06G/1.06G [00:12<00:00, 97.4MB/s]
100% 1.06G/1.06G [00:12<00:00, 89.7MB/s]


In [ ]:
!kaggle datasets download -d rishab260/uta-reallife-drowsiness-dataset -f "Fold1_part1/Fold1_part1/01/10.mov"


Dataset URL: https://www.kaggle.com/datasets/rishab260/uta-reallife-drowsiness-dataset
License(s): CC0-1.0
 98% 430M/440M [00:03<00:00, 139MB/s]
100% 440M/440M [00:03<00:00, 123MB/s]


In [ ]:
!pip install kaggle


# FINAL CODE

In [ ]:
def eye_aspect_ratio(eye):
    x = [point.x for point in eye]
    y = [point.y for point in eye]
    A = np.linalg.norm(np.array([x[1] - x[5], y[1] - y[5]]))
    B = np.linalg.norm(np.array([x[2] - x[4], y[2] - y[4]]))
    C = np.linalg.norm(np.array([x[0] - x[3], y[0] - y[3]]))
    ear = (A + B) / (2.0 * C)
    return ear

In [ ]:
def pupil_to_eye_center_distance(eye):
    x = [point.x for point in eye]
    y = [point.y for point in eye]
    d = np.linalg.norm(np.array([x[0] - x[3], y[0] - y[3]]))
    return d

In [ ]:
def mouth_aspect_ratio(mouth):
    x = [point.x for point in mouth]
    y = [point.y for point in mouth]
    A = np.linalg.norm(np.array([x[13] - x[19], y[13] - y[19]]))
    B = np.linalg.norm(np.array([x[14] - x[18], y[14] - y[18]]))
    C = np.linalg.norm(np.array([x[15] - x[17], y[15] - y[17]]))
    mar = (A + B + C) / (3.0 * np.linalg.norm(np.array([x[12] - x[16], y[12] - y[16]])))
    return mar

In [ ]:
def mouth_to_eye_ratio(eye, mouth):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(mouth)
    if ear == 0:
        return 0
    moe = mar / ear
    return moe

# FINAL CODE

In [54]:
import cv2
import dlib
import numpy as np
import zipfile
import os
import pickle

In [55]:
def determine_drowsiness(segment_features):
    drowsy_frames = 0
    threshold_ear = 0.28  # Adjusted EAR threshold
    threshold_mar = 0.5   # Adjusted MAR threshold
    consecutive_threshold = 15  # Minimum consecutive frames required for drowsiness detection

    consecutive_drowsy_count = 0

    for features in segment_features:
        ear, puc, mar, moe = features
        ear_status = "Below" if ear < threshold_ear else "Above"
        mar_status = "Above" if mar > threshold_mar else "Below"

        print(f"EAR: {ear:.2f} ({ear_status} threshold), MAR: {mar:.2f} ({mar_status} threshold)")

        if ear < threshold_ear:
            consecutive_drowsy_count += 1
            if consecutive_drowsy_count >= consecutive_threshold:
                drowsy_frames += 1
        else:
            consecutive_drowsy_count = 0  # Reset if EAR goes above threshold

    return 1 if drowsy_frames > len(segment_features) / 3 else 0  # Adjusted to make it more reliable

In [56]:
def process_video_segments(video_path, detector, predictor, frames_per_segment=100, num_segments=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < frames_per_segment:
        cap.release()
        return []

    segment_positions = np.linspace(0, total_frames - frames_per_segment, num_segments, dtype=int)
    features_segments = []
    labels_segments = []

    for i, start_pos in enumerate(segment_positions):
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_pos)
        current_segment = []

        for _ in range(frames_per_segment):
            ret, frame = cap.read()
            if not ret:
                break
            features = extract_features(frame, detector, predictor)
            current_segment.append(features)

        if len(current_segment) == frames_per_segment:
            print(f"\nSegment {i+1} Analysis:")
            segment_label = determine_drowsiness(current_segment)
            print(f"Segment {i+1} Result: {'Drowsy' if segment_label == 1 else 'Not Drowsy'}\n")
            features_segments.append(current_segment)
            labels_segments.append(segment_label)

    cap.release()
    return features_segments, labels_segments


In [57]:
# Load Dlib face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [58]:
# Unzip and extract the video
zip_file_path = "/content/10.MOV.zip"
extracted_folder_path = '/tmp/extracted_video/'

In [59]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

video_file_path = os.path.join(extracted_folder_path, '10.MOV')
output_file = "drowsiness_features.pkl"

In [60]:
all_features = []
all_labels = []

In [61]:
if os.path.exists(video_file_path):
    print(f"Processing video: {video_file_path}")
    segments, labels = process_video_segments(video_file_path, detector, predictor)
    all_features.extend(segments)
    all_labels.extend(labels)

    print(f"Total segments collected: {len(all_features)}")
    for i, label in enumerate(all_labels):
        print(f"Segment {i+1}: {'Drowsy' if label == 1 else 'Not Drowsy'}")

    data = {'features': np.array(all_features), 'labels': np.array(all_labels)}
    with open(output_file, 'wb') as f:
        pickle.dump(data, f)

    print(f"Features and labels saved to {output_file}")
else:
    print(f"Video file not found at {video_file_path}")

Processing video: /tmp/extracted_video/10.MOV

Segment 1 Analysis:
EAR: 0.27 (Below threshold), MAR: 0.04 (Below threshold)
EAR: 0.27 (Below threshold), MAR: 0.03 (Below threshold)
EAR: 0.26 (Below threshold), MAR: 0.03 (Below threshold)
EAR: 0.26 (Below threshold), MAR: 0.04 (Below threshold)
EAR: 0.25 (Below threshold), MAR: 0.03 (Below threshold)
EAR: 0.28 (Above threshold), MAR: 0.03 (Below threshold)
EAR: 0.27 (Below threshold), MAR: 0.03 (Below threshold)
EAR: 0.28 (Above threshold), MAR: 0.02 (Below threshold)
EAR: 0.27 (Below threshold), MAR: 0.04 (Below threshold)
EAR: 0.28 (Below threshold), MAR: 0.03 (Below threshold)
EAR: 0.28 (Below threshold), MAR: 0.03 (Below threshold)
EAR: 0.30 (Above threshold), MAR: 0.03 (Below threshold)
EAR: 0.32 (Above threshold), MAR: 0.04 (Below threshold)
EAR: 0.27 (Below threshold), MAR: 0.04 (Below threshold)
EAR: 0.29 (Above threshold), MAR: 0.03 (Below threshold)
EAR: 0.30 (Above threshold), MAR: 0.04 (Below threshold)
EAR: 0.28 (Above thre